In [1]:
from transformers import pipeline
model_name="paust/pko-t5-base"

def generate_text(pipe, text, num_return_sequences=5, max_length=200):
  text = f"{text}"
  out = pipe(text, num_return_sequences=num_return_sequences, max_length=max_length)
  return [x['generated_text'] for x in out]

def run_pipeline(model_path, text):
    nlg_pipeline = pipeline('text2text-generation', model=model_path, tokenizer=model_name, device='cpu')
    return (generate_text(nlg_pipeline, text, num_return_sequences=1, max_length=200)[0])

c:\Users\KHU\anaconda3\envs\tagging\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import pandas as pd
test=pd.read_csv(r"C:\Users\KHU\Desktop\tagging_seq2seq\test_df_score_drop.csv")

In [3]:
test

,sentence,tag_info
0,깔끔한 하얀색 추천드립니다,깔끔한 하얀색 추천드립니다 : (색감[2])(긍정[2])
1,솔직히 말하면 정말 가볍고,솔직히 말하면 정말 가볍고 : (무게[3])(긍정[3])
2,점점 갈수록 워치가 충전이 안되요,점점 갈수록 워치가 충전이 안되요 : (충전[3])(부정[2])
3,"이 배터리 구매의 결정적인 동기는 저전력 충전인데, 아직은 차이를 못느끼겠네요 ...","이 배터리 구매의 결정적인 동기는 저전력 충전인데, 아직은 차이를 못느끼겠네요 ..."
4,메탈제질이라 부들부들해서 기스걱정 있었는데,메탈제질이라 부들부들해서 기스걱정 있었는데 : (재질[3])(부정[2])
...,...,...
1273,디자인도 아주 깔끔하고,디자인도 아주 깔끔하고 : (디자인[1])(긍정[2])
1274,4고속충전 포트에서 충전하는데 고속충전이 안될때가있음 해결은 usb를 뽑고 다시 ...,4고속충전 포트에서 충전하는데 고속충전이 안될때가있음 해결은 usb를 뽑고 다시 ...
1275,충전속도는 좀 느려요,충전속도는 좀 느려요 : (충전[2])(부정[2])
1276,베터리 잔량이 표시되어 이용 편리성이 좋아요,베터리 잔량이 표시되어 이용 편리성이 좋아요 : (잔량표시[3])(긍정[2])


In [4]:
import re

def extract_element(sentence):
    # 정규 표현식 패턴
    pattern = r"\(\s*(?P<topic>.+?)\s*\[\s*(?P<topic_score>\d+)\s*\]\s*\)\s*\(\s*(?P<sentiment>.+?)\s*\[\s*(?P<sentiment_score>\d+)\s*\]\s*\)"

    # 정규식 컴파일
    regex = re.compile(pattern)
    result={}
    tag_if=False
    # 매칭 및 값 추출
    match = regex.search(sentence)
    if match:
        topic = match.group('topic')
        topic_score = match.group('topic_score')
        sentiment = match.group('sentiment')
        sentiment_score = match.group('sentiment_score')
        result['topic']=topic
        result['topic_score']=topic_score
        result['sentiment']=sentiment
        result['sentiment_score']=sentiment_score
        tag_if=True

    return tag_if, result

In [5]:
topic_origin=[]
topic_score_origin=[]
sentiment_origin=[]
sentiment_score_origin=[]
topic_pred=[]
topic_score_pred=[]
sentiment_pred=[]
sentiment_score_pred=[]

for index, row in test.iterrows():
    tag_info, result=extract_element(row['tag_info'])
    if tag_info:
        topic_origin.append(result['topic'])
        topic_score_origin.append(result['topic_score'])
        sentiment_origin.append(result['sentiment'])
        sentiment_score_origin.append(result['sentiment_score'])
    else:
        topic_origin.append("X")
        topic_score_origin.append("X")
        sentiment_origin.append("X")
        sentiment_score_origin.append("X")

    tag_info, result=extract_element(run_pipeline('with_score_model_drop.pth', row['sentence']))
    if tag_info:
        topic_pred.append(result['topic'])
        topic_score_pred.append(result['topic_score'])
        sentiment_pred.append(result['sentiment'])
        sentiment_score_pred.append(result['sentiment_score'])
    else:
        topic_pred.append("X")
        topic_score_pred.append("X")
        sentiment_pred.append("X")
        sentiment_score_pred.append("X")

In [6]:
from sklearn.metrics import classification_report
def label_encoding(original_labels, predicted_labels):
    all_labels = set(original_labels) | set(predicted_labels)
    label_to_int = {label: idx for idx, label in enumerate(all_labels)}
    int_to_label = {idx: label for label, idx in label_to_int.items()}
    return [label_to_int[label] for label in original_labels], [label_to_int[label] for label in predicted_labels], int_to_label

# 정수형 레이블로 변환
topic_origin_encoded, topic_pred_encoded, topic_label_map = label_encoding(topic_origin, topic_pred)
topic_score_origin_encoded, topic_score_pred_encoded, topic_score_label_map = label_encoding(topic_score_origin, topic_score_pred)
sentiment_origin_encoded, sentiment_pred_encoded, sentiment_label_map = label_encoding(sentiment_origin, sentiment_pred)
sentiment_score_origin_encoded, sentiment_score_pred_encoded, sentiment_score_label_map = label_encoding(sentiment_score_origin, sentiment_score_pred)

# 분류 보고서 출력
print("Topic Classification Report:")
print(classification_report(topic_origin_encoded, topic_pred_encoded, target_names=list(topic_label_map.values())))
print("Topic Score Classification Report:")
print(classification_report(topic_score_origin_encoded, topic_score_pred_encoded, target_names=list(topic_score_label_map.values())))

print("Sentiment Classification Report:")
print(classification_report(sentiment_origin_encoded, sentiment_pred_encoded, target_names=list(sentiment_label_map.values())))
print("Sentiment Score Classification Report:")
print(classification_report(sentiment_score_origin_encoded, sentiment_score_pred_encoded, target_names=list(sentiment_score_label_map.values())))


Topic Classification Report:
              precision    recall  f1-score   support

        동시충전       0.67      0.40      0.50        10
        잔량표시       0.80      0.88      0.83        40
        고속충전       0.85      0.82      0.83        65
        충전표시       0.67      0.57      0.62        14
        휴대성과       0.00      0.00      0.00         0
         저전력       0.83      0.62      0.71         8
          as       0.00      0.00      0.00         0
         과전류       0.00      0.00      0.00         1
         구성품       0.50      0.14      0.22         7
         케이블       0.73      0.80      0.76        30
    배송/포장/발송       0.86      0.96      0.91       110
          무게       0.93      0.99      0.96       143
          안전       0.67      0.33      0.44         6
         사이즈       0.88      0.90      0.89        94
        맥세이프       0.58      0.58      0.58        19
          AS       1.00      0.38      0.55         8
       디스플레이       0.88      0.70      0.78        1

c:\Users\KHU\anaconda3\envs\tagging\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\KHU\anaconda3\envs\tagging\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\KHU\anaconda3\envs\tagging\lib\site-packages\sklearn\metrics\_classification.py:1517: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\KHU\an

TOOD
한 문장에 태깅이 여러개 있는 경우 처리 필요함

접착력은 엄청 강합니다 : (부착[3])(긍정[3]) // 케이스 미착용 기준으로 접착력은 엄청 강합니다 : (부착[3])(긍정[2])

이런 경우 절반만 맞거나 하는 경우 어떻게 loss값을 결정할건지

In [7]:
# topic_pred=[]
# topic_score_pred=[]
# sentiment_pred=[]
# sentiment_score_pred=[]

# for index, row in test.iloc[:10].iterrows():
#     print("원본:", row['sentence'])
#     print("실제 태깅:",row['tag_info'])
#     extract_element(row['tag_info'])
#     print("모델 태깅:",run_pipeline('with_score_model.pth', row['sentence']))
#     extract_element(run_pipeline('with_score_model.pth', row['sentence']))
#     print()